# Mount the Gdrive


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)


Mounted at /content/gdrive


In [ ]:
!ls "/content/gdrive/My Drive/eva4p2/drone-dataset.zip"

'/content/gdrive/My Drive/eva4p2/drone-dataset.zip'


#Copy the Folder to Colab for Training

In [ ]:
cd /content

/content


In [5]:
!ls

drone-dataset  processed-drone-dataset	sample_data
gdrive	       raw-drone-dataset.zip	session2-dataset-notused


In [ ]:
cp -r "/content/gdrive/My Drive/eva4p2/drone-dataset.zip" "raw-drone-dataset.zip"

In [ ]:
!unzip raw-drone-dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: drone-dataset/train/3/1d78780c-Sentera-Phoenix-Fixed-Wing-Drone-thumb.jpg  
  inflating: drone-dataset/train/3/Image_331.jpg  
  inflating: drone-dataset/train/3/screen-shot-2017-04-06-at-16-45-00.jpg  
  inflating: drone-dataset/train/3/1-Figure1-1.jpg  
  inflating: drone-dataset/train/3/images (250).jpg  
  inflating: drone-dataset/train/3/image145.jpg  
  inflating: drone-dataset/train/3/images (227).jpg  
  inflating: drone-dataset/train/3/Tigerwing2BVTOL2BMapping.jpg  
  inflating: drone-dataset/train/3/uav-lidar-DT26X-880x470.jpg  
  inflating: drone-dataset/train/3/Image_318.jpg  
  inflating: drone-dataset/train/3/best-fixed-wing-drone-RC-Nano-Skyhunter-PNP.jpg  
  inflating: drone-dataset/train/3/5bee3ab23a641.jpg  
  inflating: drone-dataset/train/3/RussianDroneKamikaze (1).jpg  
  inflating: drone-dataset/train/3/Image_194.jpg  
  inflating: drone-dataset/train/3/sandhu-eva4p2-imgg0125.jpg  
  inflating: drone-

#Preprocess the Files, remove the waste, uncompatible files etc and calculate mean and std 

In [ ]:
import numpy as np
import cv2
import io
import os
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import ToTensor
from torchvision import transforms
import zipfile
from PIL import Image
import sys
import torch.utils.data
import timeit
import logging

# This is the common class for both test and train data sets. Based on indexes, this class serves the transformed image tensor.
  
#Considered 4 folders (n). Take all the files from those folders into a list, a global list in memory.     
#Flying birds, Large Quadcopters, Small QuadCopters, Winged drones
#Given an index, list[index] will be returned as a file.
#root_dir = "/content/drone-dataset"
class PreprocessImages(Dataset):

  def __init__(self, root_dir, processed_root_dir, transform=ToTensor()):
    'Initialization'
    self.m_root = root_dir
    self.m_transform = transform
    logging.basicConfig(filename = "/content/dataloader_timeit.log",datefmt='%H:%M:%S',level=logging.DEBUG)
    #self.g_dict_fileindex = dict()
    self.g_array_fileindex1 = []
    self.m_processed_root = processed_root_dir
    self.walkTheDirectory(self.m_root) 
    self.m_length = len(self.g_array_fileindex1)
  
  #root_dir structure
  #train
  #   /folder1
  #     file.jpg
  #     file40.jpg  
  #   /folder2
  #     filex.jpg
  #     filey.40.jpg  
  #   /folder3
  #     filea.jpg
  #     fileb.40.jpg  
  #   /folder4
  #     fileaa.jpg
  #     filebax.jpg  
  
  #test
  #   /anyfoldername1
  #     sfile.jpg
  #     mfile40.jpg  
  #   /foldername2
  #     efilex.jpg
  #     cfiley.40.jpg  
  #   /foldername3
  #     somefilea.jpg
  #     vfileb.40.jpg  
  #   /foldername4
  #     somefileaa.jpg
  #     afilebax.jpg  
  
  
    
  def walkTheDirectory(self,root_dir):
    # for file in files:
    #    with open(os.path.join(root, file), "r") as auto: 
    subdirs = [x[0] for x in os.walk(root_dir)]
    dir = ''
    try:
      dir = self.m_processed_root
      os.mkdir(dir)
    except:
      print('dir present',self.m_processed_root)

    count = -1
    dir1 =  self.m_processed_root
    print('dir1=',dir1) 
    for d in subdirs:
      count = count + 1
      if count == 0:
        continue
      basedir = os.path.basename(subdirs[count])
      dir = os.path.join(dir1,basedir)
      print(dir)
      try:
        os.mkdir(dir)
      except:
        print('dir present', dir)
    
    for path, subdirs, files in os.walk(self.m_root): # path should give the complete path till the last directory.
        for filename in files:
            #print(os.path.join(path, filename))
            full_path = os.path.join(path, filename)
            #self.g_dict_fileindex[index] = full_path
            self.g_array_fileindex1.append(full_path)
            
 
  def __len__(self):
    'Denotes the total number of samples'
    return self.m_length;

  def printItem(self,idx):
    if idx >= self.m_length:
        print('index', idx, 'is bigger than the actual dataset size')
        return
    file = self.g_array_fileindex1[idx]
    print('printFile-',file)  
  
  def removeItem(self,idx):
    if idx >= self.m_length:
        print('index', idx, 'is bigger than the actual dataset size')
        return
    #self.g_array_fileindex.remove(file)

  def __getitem__(self, idx):
    if idx >= self.m_length:
        print('index', idx, 'is bigger than the actual dataset size')
        return
        
    start_time = timeit.default_timer()

    # Pick the file from the index
    size = 224, 224
    file = self.g_array_fileindex1[idx]
    
    """
    try:
      im = Image.open(file)
      im.resize(size)
      im.save(save)
    except IOError:
      print("cannot create thumbnail for", file)
    """
    #new_dir = m_processed_root
    file1 = file
    try:
      try:
        image = Image.open(file).convert('RGB') # even grey scale images will be converted to RGB with 3 channels
      except:
        raise "IOError"
      #temporarily we will resize to 224*224
      # We need to look into meaningful way of resizing (...pyramid pooling??)
      width,height = image.size
      #print('width=',width,'height=',height)
      image = image.resize((224,224), Image.BICUBIC)
      #print('image converted with BICUBIC')
      filename, file_extension = os.path.splitext(file)
      basedir = os.path.basename(os.path.dirname(filename)) # 0,1,2,3 dirs of either train or test
      #append subdir to new dir
      dir1 = os.path.join(self.m_processed_root,basedir) # This is new directory path.
    
      #Create jpg file
      basefile=os.path.basename(file)#This comes with extension
      #split with extension
      file1, ext = os.path.splitext(basefile)
      file1 = file1 + ".jpg"
      file1 = os.path.join(dir1,file1)#complete new file path
      #print('new filename=',file1)
      try:
        image.save(file1) # incase if it is already present
        #print('post proprocessed filename=',file1)
      except:
        print('post proprocessed filename=',file1)
        
      if self.m_transform:
        #print('transform to tensor')
        image = self.m_transform(image)
      
      self.g_array_fileindex1[idx] = file1
    except:
      print("exception while opening=",file)
      
    load_time = timeit.default_timer() - start_time
    
    desc = f' file={file1} orig_width={width},orig_height={height}, PROCESSING_TIME={load_time:0.3f}'
    logging.info(desc)
      
    return image
"""   
#import WingsFolderDataset as wingsDataset
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

train_root_dir = "/content/session2-dataset/train"
processed_train_root_dir = "/content/processed-drone-dataset/train"

tr = transforms.Compose([
    #transforms.RandomHorizontalFlip(),
    #transforms.RandomCrop(),
    transforms.ToTensor()
    #normalize,
    #transforms.RandomErasing()
])

train_data_set = PreprocessImageFolderDataset(train_root_dir, processed_train_root_dir, transform = tr)

means = []
stds = []
print('length of dataset=',len(train_data_set))
for i in range(len(train_data_set)):
    if i == 1000 or i or 5000 or i == 10000 or i == 15000 or i == 20000:
      print('i=',i)
    
    try:
      img  = train_data_set[i]
      means.append(torch.mean(img))
      stds.append(torch.std(img))
    except:
      train_data_set.printItem(i)
      
    #print(torch.mean(img))
    
mean = torch.mean(torch.tensor(means))
std = torch.mean(torch.tensor(stds))

print('mean=',mean)
print('std=',std)
"""


#Calculate Mean and Std of resized files

In [16]:
import torch
import torchvision.transforms

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

#Assuming test data represent the full set of train data. So we use normalized values of traion data in test data also.

train_root_dir = "/content/drone-dataset/train"
processed_train_root_dir = "/content/processed-drone-dataset/train"

tr = transforms.Compose([
    #transforms.RandomHorizontalFlip(),
    #transforms.RandomCrop(),
    transforms.ToTensor()
    #normalize,
    #transforms.RandomErasing()
])

train_data_set = PreprocessImages(train_root_dir, processed_train_root_dir, transform = tr)

def mean_std(dataset):
  dataloader = torch.utils.data.DataLoader(dataset, batch_size=1,
                                          shuffle=False, num_workers=1)

  mean = torch.zeros(3)
  std = torch.zeros(3)
  print('Computing mean and std from dataloader')
  for inputs in dataloader:
	  for i in range(3):
		  mean[i] = mean[i] + inputs[:,i,:, :].mean()
		  std[i]  = std[i] + inputs[:,i,:, :].std()
  mean.div_(len(dataset))
  std.div_(len(dataset))
  print('mean=',mean)
  print('std=',std)
  return mean, std

print('calculating mean,std of train dataset')
mean_std(train_data_set)

test_root_dir = "/content/drone-dataset/test"
processed_test_root_dir = "/content/processed-drone-dataset/test"

test_data_set = PreprocessImages(test_root_dir, processed_test_root_dir, transform = tr)

print('calculating mean,std of test dataset')
mean_std(test_data_set)


dir present /content/processed-drone-dataset/train
dir1= /content/processed-drone-dataset/train
/content/processed-drone-dataset/train/2
dir present /content/processed-drone-dataset/train/2
/content/processed-drone-dataset/train/3
dir present /content/processed-drone-dataset/train/3
/content/processed-drone-dataset/train/.ipynb_checkpoints
dir present /content/processed-drone-dataset/train/.ipynb_checkpoints
/content/processed-drone-dataset/train/1
dir present /content/processed-drone-dataset/train/1
/content/processed-drone-dataset/train/0
dir present /content/processed-drone-dataset/train/0
calculating mean,std of train dataset
Computing mean and std from dataloader
mean= tensor([0.5264, 0.5791, 0.6111])
std= tensor([0.1701, 0.1641, 0.1792])
dir present /content/processed-drone-dataset/test
dir1= /content/processed-drone-dataset/test
/content/processed-drone-dataset/test/2
dir present /content/processed-drone-dataset/test/2
/content/processed-drone-dataset/test/3
dir present /content

(tensor([0.5242, 0.5772, 0.6061]), tensor([0.1700, 0.1646, 0.1822]))

#Save the Preprocessed dataset

In [17]:
cp -r "/content/processed-drone-dataset" "/content/gdrive/My Drive/processed-drone-dataset"

In [21]:
!zip -r processed-drone-dataset.zip /content/processed-drone-dataset  

Streaming output truncated to the last 5000 lines.
  adding: content/processed-drone-dataset/train/0/image169.jpg (deflated 2%)
  adding: content/processed-drone-dataset/train/0/image (75).jpg (deflated 18%)
  adding: content/processed-drone-dataset/train/0/ANd9GcREQK6e0H-fMUEaRKwZbw-dRPOJdsZC1J8Qow.jpg (deflated 16%)
  adding: content/processed-drone-dataset/train/0/XBt0ucY6_-E.jpg (deflated 9%)
  adding: content/processed-drone-dataset/train/0/exotic-flying-bird-is-isolated-on-a-black-background-KDA8RM.jpg (deflated 9%)
  adding: content/processed-drone-dataset/train/0/image711.jpg (deflated 3%)
  adding: content/processed-drone-dataset/train/0/image43.jpg (deflated 1%)
  adding: content/processed-drone-dataset/train/0/040a2293b71be6b3abe4a2a04b89a763.jpg (deflated 2%)
  adding: content/processed-drone-dataset/train/0/E_9s8euT_fpSNc39vMaJIAHaEi.jpg (deflated 2%)
  adding: content/processed-drone-dataset/train/0/images173 (1).jpg (deflated 19%)
  adding: content/processed-drone-datase

In [30]:
cp -r "/content/processed-drone-dataset.zip" "/content/gdrive/My Drive/eva4p2/."

# Create Custom Dataset Class to be used for Test and Train samples

In [ ]:
import numpy as np
import cv2
import io
import os
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import ToTensor
from torchvision import transforms
import zipfile
from PIL import Image
import sys
import torch.utils.data
import timeit
import logging

# This is the common class for both test and train data sets. Based on indexes, this class serves the transformed image tensor.
  
#Considered 4 folders (n). Take all the files from those folders into a list, a global list in memory.     
#Flying birds, Large Quadcopters, Small QuadCopters, Winged drones
#Given an index, list[index] will be returned as a file.
class WingsDataset(Dataset):

  def __init__(self, root_dir, transform=ToTensor()):
    'Initialization'
    self.m_root = root_dir
    self.m_transform = transform
    logging.basicConfig(filename = "/content/dataloader_timeit1.log",datefmt='%H:%M:%S',level=logging.DEBUG)
    #self.g_dict_fileindex = dict()
    self.g_array_fileindex = []
    self.walkTheDirectory(self.m_root) 
    self.m_length = len(self.g_array_fileindex)
  
  #root_dir structure
  #train
  #   /folder1
  #     file.jpg
  #     file40.jpg  
  #   /folder2
  #     filex.jpg
  #     filey.40.jpg  
  #   /folder3
  #     filea.jpg
  #     fileb.40.jpg  
  #   /folder4
  #     fileaa.jpg
  #     filebax.jpg  
  
  #test
  #   /anyfoldername1
  #     sfile.jpg
  #     mfile40.jpg  
  #   /foldername2
  #     efilex.jpg
  #     cfiley.40.jpg  
  #   /foldername3
  #     somefilea.jpg
  #     vfileb.40.jpg  
  #   /foldername4
  #     somefileaa.jpg
  #     afilebax.jpg  
  
  
    
  def walkTheDirectory(self,root_dir):
    #for root, dirs, files in os.walk('python/Lib/email'):
    # for file in files:
    #    with open(os.path.join(root, file), "r") as auto:    
    for path, subdirs, files in os.walk(self.m_root): # path should give the complete path till the last directory.
        for filename in files:
            #print(os.path.join(path, filename))
            full_path = os.path.join(path, filename)
            #self.g_dict_fileindex[index] = full_path
            self.g_array_fileindex.append(full_path)
            
 
  def __len__(self):
    'Denotes the total number of samples'
    return self.m_length;

  def printItem(self,idx):
    if idx >= self.m_length:
        print('index', idx, 'is bigger than the actual dataset size')
        return
    file = self.g_array_fileindex[idx]
    print('printItem-',file)  

  def __getitem__(self, idx):

    if idx >= self.m_length:
        print('index', idx, 'is bigger than the actual dataset size')
        return
        
    start_time = timeit.default_timer()

    # Pick the file from the index
    file = self.g_array_fileindex[idx]
    
    image = Image.open(file).convert('RGB') # even grey scale images will be converted to RGB with 3 channels
    w,h = image.size
    image = image.resize((224,224)) # These are already resized images. Dont do preprocessing here..
    
    if self.m_transform:
      image = self.m_transform(image)
    
    load_time = timeit.default_timer() - start_time
    
    desc = f' file={file} original_w={w},original_h={h} LOAD_TIME={load_time:0.3f}'
    logging.info(desc)
      
    return image
    

#Train/Test loader

In [ ]:

import torch
import torchvision.transforms

#import WingsFolderDataset as wingsDataset
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

root_dir = "/content/Session2Dataset-copy"
# replace with actual mean and std values
normalize = transforms.Normalize(mean=[149.17579724, 143.51813416, 136.34473418],
                                        std=[10.918, 10.54722837, 9.7497292])

tr = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(),
    transforms.ToTensor()
    normalize,
    transforms.RandomErasing()
])

train_set = WingsFolderDataset("/content/Session2Dataset-train", transform = tr)



train_loader = torch.utils.data.DataLoader(train_set, batch_size=64,
                                          shuffle=True, num_workers=1,pin_memory=False)

test_transform = transforms.Compose([transforms.ToTensor(),normalize])
test_set = WingsFolderDataset("/content/Session2Dataset-test", transform = tr)('/content/Dataset_224/', size = 100000, test=True, transform = test_transform)

test_loader = torch.utils.data.DataLoader(test_set, batch_size=64, shuffle=False, num_workers=1)


# Train, Test utility functions

In [ ]:
from __future__ import print_function
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR
import torch.optim as optim
import copy
from torchsummary import summary
from torchvision import datasets, transforms
import numpy as np

def train_model(model, device, train_loader, optimizer, epoch,train_losses,train_acc,criteria, store_mode ='epoch', doL1 = 0,doL2 = 0,LAMBDA = 0):
  print('L1=',doL1,';L2=',doL2,';LAMBDA=',LAMBDA,'epoch=',epoch)
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):
    # get samples
    data, target = data.to(device), target.to(device)
    #print('data=',len(data),';target=',len(target))

    # Init
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes. 
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

    # Predict
    y_pred = model(data)

    # Calculate loss
    #print('y_pred=',len(y_pred.dataset),'target=',len(target.dataset))
    #loss = F.nll_loss(y_pred, target)
    #criteria = nn.CrossEntropyLoss()
    loss = criteria(y_pred, target) 
    reg_loss=0
    if (doL1 == 1):
      for p in model.parameters():  
        reg_loss += torch.sum(torch.abs(p.data))
    if (doL2 == 1):
      for p in model.parameters():
        reg_loss += torch.sum(p.data.pow(2))    
    
    loss+=LAMBDA*reg_loss
    
    train_losses.append(loss)

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Update pbar-tqdm
    
    pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)
    if store_mode == 'mini_batch':  # Store loss and accuracy
        batch_accuracy = 100 * correct / processed
        if not train_losses is None:
            train_losses.append(loss.item())
        if not train_acc is None:
            train_acc.append(batch_accuracy)
        
    if store_mode == 'epoch':   # Store loss and accuracy
        accuracy = 100 * correct / processed
        if not train_losses is None:
            train_losses.append(loss.item())
        if not accuracies is None:
            train_acc.append(accuracy)

    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    #train_acc.append(100*correct/processed)


def test_model(model, device, test_loader,test_losses,test_acc,criteria, correct_samples, incorrect_samples, sample_count=30, last_epoch=False):
    model.eval()
    test_loss = 0
    correct = 0
    #criteria = nn.CrossEntropyLoss()
            
    with torch.no_grad():
        for data, target in test_loader:
            img_batch = data
            data, target = data.to(device), target.to(device)
            #print('data=',len(data),';target=',len(target))
            output = model(data)
            #test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            #test_loss += criteria(output, target).item()
            test_loss += criteria(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            result = pred.eq(target.view_as(pred))
            if last_epoch:
                #print('last_epoch=',last_epoch)
                for i in range(len(list(result))):
                    if not list(result)[i] and len(incorrect_samples) < sample_count:
                        incorrect_samples.append({
                            'prediction': list(pred)[i],
                            'label': list(target.view_as(pred))[i],
                            'image': img_batch[i]
                            
                        })
                    elif list(result)[i] and len(correct_samples) < sample_count:
                        correct_samples.append({
                            'prediction': list(pred)[i],
                            'label': list(target.view_as(pred))[i],
                            'image': img_batch[i]
                            
                        })
            correct += result.sum().item()
            #correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), 
        100. * correct / len(test_loader.dataset)))
    test_acc.append(100. * correct / len(test_loader.dataset)) 
    return test_loss
#Global functions
def show_summary(model,input_size = (3, 224, 224)):
    summary(model.m_model, input_size)
    
def run_model(model, device, train_loader, optimizer, train_losses, train_acc, test_loader, test_losses, test_acc,  criteria = F.nll_loss, doL1 = 0, doL2 = 0, LAMBDA = 0, EPOCHS = 20,start=0):
    #scheduler = StepLR(optimizer, step_size=8, gamma=0.1)
    for epoch in range(EPOCHS):
        print("EPOCH:", (start+epoch))
        train_model(model, device, train_loader, optimizer, epoch,train_losses,train_acc,criteria,doL1,doL2,LAMBDA)
        test_model(model, device, test_loader,test_losses,test_acc,criteria)





In [ ]:
import torch
import torchvision.transforms

#import WingsFolderDataset as wingsDataset
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

root_dir = "/content/gdrive/My\ Drive/Session 2 Dataset"

train_transform = transforms.Compose([
    #transforms.RandomHorizontalFlip(),
    #transforms.RandomCrop(),
    transforms.ToTensor()
    #normalize,
    #transforms.RandomErasing()
])

data_set = WingsFolderDataset(root_dir, transform = train_transform)

means = []
stds = []
for img in subset(dataset):
    means.append(torch.mean(img))
    stds.append(torch.std(img))

mean = torch.mean(torch.tensor(means))
std = torch.mean(torch.tensor(stds))

print('mean=',mean)
print('std=',std)

#normalize = transforms.Normalize(mean=[149.17579724, 143.51813416, 136.34473418],
#                                        std=[10.918, 10.54722837, 9.7497292])


train_loader = torch.utils.data.DataLoader(data_set, batch_size=64,
                                          shuffle=True, num_workers=1,pin_memory=False)

test_transform = transforms.Compose([transforms.ToTensor(),normalize])
#test_set = dt.MonacularDepthEstimation_Dataset('/content/Dataset_224/', size = 100000, test=True, transform = valid_transform)

#val_loader = torch.utils.data.DataLoader(test_set, batch_size=1, shuffle=False, num_workers=1,pin_memory=True)

TypeError: ignored

### This notebook is optionally accelerated with a GPU runtime.
### If you would like to use this acceleration, please select the menu option "Runtime" -> "Change runtime type", select "Hardware Accelerator" -> "GPU" and click "SAVE"

----------------------------------------------------------------------

# MobileNet v2

*Author: Pytorch Team*

**Efficient networks optimized for speed and memory, with residual blocks**

_ | _
- | -
![alt](https://pytorch.org/assets/images/mobilenet_v2_1.png) | ![alt](https://pytorch.org/assets/images/mobilenet_v2_2.png)

In [ ]:
import torch
#define the model
model = torch.hub.load('pytorch/vision:v0.6.0', 'mobilenet_v2', pretrained=True)
model.eval()

In [ ]:
#trace model with a dummy input
traced_model = torch.jit.trace(model, torch.randn(1,3,224,224))
traced_model.save('mobilenetV2.pt')

All pre-trained models expect input images normalized in the same way,
i.e. mini-batches of 3-channel RGB images of shape `(3 x H x W)`, where `H` and `W` are expected to be at least `224`.
The images have to be loaded in to a range of `[0, 1]` and then normalized using `mean = [0.485, 0.456, 0.406]`
and `std = [0.229, 0.224, 0.225]`.

Here's a sample execution.

In [ ]:
# Download an example image from the pytorch website
import urllib
url, filename = ("https://github.com/pytorch/hub/raw/master/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

In [ ]:
# sample execution (requires torchvision)
from PIL import Image
from torchvision import transforms
input_image = Image.open(filename)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)
# Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
print(output[0])
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
print(torch.nn.functional.softmax(output[0], dim=0))


### Model Description

The MobileNet v2 architecture is based on an inverted residual structure where the input and output of the residual block are thin bottleneck layers opposite to traditional residual models which use expanded representations in the input. MobileNet v2 uses lightweight depthwise convolutions to filter features in the intermediate expansion layer. Additionally, non-linearities in the narrow layers were removed in order to maintain representational power.

| Model structure | Top-1 error | Top-5 error |
| --------------- | ----------- | ----------- |
|  mobilenet_v2       | 28.12       | 9.71       |


### References

 - [MobileNetV2: Inverted Residuals and Linear Bottlenecks](https://arxiv.org/abs/1801.04381)